In [1]:
library("xgboost")
library("Matrix")
library('Ckmeans.1d.dp')
library('lightgbm')



Attaching package: ‘lightgbm’


The following object is masked from ‘package:xgboost’:

    slice




## Read data and process data labels

In [2]:
time_matrix <- matrix(0,ncol = 3, nrow =4)
colnames(time_matrix) <- c("user_time", "system_time", "elapsed_time")
start_time = Sys.time()

In [3]:
data=read.csv('Hourly_features_matrix.csv')
data=data[,2:(dim(data)[2]-6)]

In [4]:
dim(data)

[1] 414  21

In [5]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [6]:
dlist= load('Hourly_ets_datalist.RData')
datalist=eval(parse(text = dlist ))
res=datalist[[1]]
MASE=res[,,,6]
m=5

In [7]:
dim(MASE)

[1] 414   5   4

In [8]:
whichmin<-function(x){
    minx=min(x[x>0])
    loc=which(x==minx)[1]-1
    loc
}

meanunique=function(x)
    {
    mean(unique(x))
}

In [9]:
nanum=c()
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    a=apply(count,1,min)
    if(sum(is.na(a))>0)
        {
        nanum=append(nanum,i)
    }
    }

In [10]:
nanum

NULL

In [11]:
realbestmin=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    min_value=apply(count,1,min)
    if (max(min_value,na.rm = TRUE)==0){
        realbestmin[i,]=0
        }
    else
        {
        min_value[is.na(min_value)]=100 
        realbestmin[i,]= whichmin(min_value)
    }
    }

In [12]:
table(realbestmin)

realbestmin
  0   1   2   3   4 
108  52  65  73 116 

In [13]:
realbestmean=matrix(0,dim(MASE)[1],1)
for(i in seq(1,dim(MASE)[1]))
    {
    count=MASE[i,,]
    mean_value=apply(count,1,meanunique)
    if (max(mean_value,na.rm = TRUE)==0){
        realbestmean[i,]=0
        }
    else
        {
        mean_value[is.na(mean_value)]=100 
        realbestmean[i,]= whichmin(mean_value)
    }
    }

In [14]:
realbestmean

2
1
0
0
2
3
2
3
3
0
2


In [15]:
head(data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [16]:
set.seed(100)
index = sample(2,nrow(data),replace = TRUE,prob=c(0.7,0.3))

In [17]:
train_data=data[index==1,]
test_data=data[index==2,]
train_label_min=realbestmin[index==1,]
test_label_min=realbestmin[index==2,]
train_label_mean=realbestmean[index==1,]
test_label_mean=realbestmean[index==2,]

In [18]:
head(train_data)

,frequency,nperiods,seasonal_period,trend,spike,linearity,curvature,e_acf1,e_acf10,seasonal_strength,⋯,trough,entropy,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,length
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,24,1,24,0.8512223,4.041507e-09,5.3506983,-3.435885,0.8957711,2.150024,0.9766084,⋯,22,0.2817251,0.9662029,3.396754,0.8879107,3.045070,0.09813614,0.1914053,0.8944637,700
2,24,1,24,0.9194922,2.449422e-09,8.2809591,-1.146953,0.9141691,2.554141,0.9728383,⋯,22,0.2842604,0.9705927,3.196374,0.8890215,3.165964,0.08323323,0.1945535,0.9155878,700
3,24,1,24,0.9079559,3.241234e-09,-9.0028041,1.658264,0.9086270,2.440903,0.9656292,⋯,24,0.3864782,0.9659160,3.037510,0.8618019,2.634194,0.05203444,0.1276246,0.8720262,700
4,24,1,24,0.8712692,2.010702e-09,0.7677543,2.701202,0.9187827,2.040658,0.9830436,⋯,24,0.2338078,0.9644409,3.478424,0.9301440,3.487436,0.44839902,0.7087884,0.9048109,700
5,24,1,24,0.8801393,3.844576e-09,-0.0186239,5.952095,0.9412170,2.951870,0.9634139,⋯,24,0.2962368,0.9610718,3.048530,0.9253001,3.140156,0.45128188,0.6560024,0.8393581,700
6,24,1,24,0.8551463,3.051179e-09,-2.8143809,2.642106,0.8964147,2.388106,0.9742814,⋯,11,0.3482880,0.9609369,3.103302,0.8639313,2.380859,0.18178351,0.2069934,0.8804445,700


In [19]:
end_time = Sys.time()

In [20]:
time_matrix[1,]=end_time-start_time

## Target the interval where the actual error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_min_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_min)) 
dtrain_xg_min_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)) )

In [23]:
dtrain_lg_min_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_min))
dtrain_lg_min_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_min)))

In [24]:
xgb_min_reg <- xgboost(data = dtrain_xg_min_reg, nround=100)

[1]	train-rmse:1.772672 
[2]	train-rmse:1.470261 
[3]	train-rmse:1.226028 
[4]	train-rmse:1.034763 
[5]	train-rmse:0.948514 
[6]	train-rmse:0.858530 
[7]	train-rmse:0.762533 
[8]	train-rmse:0.713903 
[9]	train-rmse:0.660367 
[10]	train-rmse:0.587598 
[11]	train-rmse:0.550408 
[12]	train-rmse:0.515659 
[13]	train-rmse:0.468171 
[14]	train-rmse:0.441081 
[15]	train-rmse:0.395667 
[16]	train-rmse:0.368636 
[17]	train-rmse:0.353864 
[18]	train-rmse:0.333663 
[19]	train-rmse:0.311382 
[20]	train-rmse:0.293384 
[21]	train-rmse:0.275998 
[22]	train-rmse:0.252193 
[23]	train-rmse:0.225634 
[24]	train-rmse:0.211258 
[25]	train-rmse:0.198183 
[26]	train-rmse:0.184462 
[27]	train-rmse:0.176207 
[28]	train-rmse:0.166884 
[29]	train-rmse:0.156948 
[30]	train-rmse:0.143481 
[31]	train-rmse:0.136548 
[32]	train-rmse:0.120538 
[33]	train-rmse:0.104768 
[34]	train-rmse:0.098856 
[35]	train-rmse:0.092588 
[36]	train-rmse:0.085905 
[37]	train-rmse:0.082937 
[38]	train-rmse:0.077907 
[39]	train-rmse:0.072

In [25]:
xgb_min_cl <- xgboost(data = dtrain_xg_min_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.347835 
[2]	train-mlogloss:1.154139 
[3]	train-mlogloss:1.017580 
[4]	train-mlogloss:0.893221 
[5]	train-mlogloss:0.804634 
[6]	train-mlogloss:0.720709 
[7]	train-mlogloss:0.642210 
[8]	train-mlogloss:0.578436 
[9]	train-mlogloss:0.536572 
[10]	train-mlogloss:0.487418 
[11]	train-mlogloss:0.450826 
[12]	train-mlogloss:0.416660 
[13]	train-mlogloss:0.376522 
[14]	train-mlogloss:0.344939 
[15]	train-mlogloss:0.315568 
[16]	train-mlogloss:0.290141 
[17]	train-mlogloss:0.268871 
[18]	train-mlogloss:0.249287 
[19]	train-mlogloss:0.231585 
[20]	train-mlogloss:0.215679 
[21]	train-mlogloss:0.200014 
[22]	train-mlogloss:0.187748 
[23]	train-mlogloss:0.175433 
[24]	train-mlogloss:0.165387 
[25]	train-mlogloss:0.154816 
[26]	train-mlogloss:0.143769 
[27]	train-mlogloss:0.134696 
[28]	train-mlogloss:0.126970 
[29]	train-mlogloss:0.119930 
[30]	train-mlogloss:0.112613 
[31]	train-mlogloss:0.106111 
[32]	train-mlogloss:0.101014 
[33]	train-mlogloss:0.096005 
[34]	train-mlogloss

In [26]:
lgb_min_reg <- lgb.train(data = dtrain_lg_min_reg, nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021464 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1809
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 21
[LightGBM] [Info] Start training from score 2.068259
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_min_cl <- lgb.train(data = dtrain_lg_min_cl,nrounds = 100,params=params)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1809
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 21
[LightGBM] [Info] Start training from score -1.376108
[LightGBM] [Info] Start training from score -1.991293
[LightGBM] [Info] Start training from score -1.788352
[LightGBM] [Info] Start training from score -1.728929
[LightGBM] [Info] Start training from score -1.323464
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
end_time = Sys.time()
time_matrix[2,]=end_time-start_time

## Target the interval where the average error is minimum

In [21]:
start_time = Sys.time()

In [22]:
dtrain_xg_mean_reg <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(train_label_mean)) 
dtrain_xg_mean_cl <- xgb.DMatrix(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)) )

In [23]:
dtrain_lg_mean_reg <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(train_label_mean))
dtrain_lg_mean_cl <- lgb.Dataset(data = as.matrix(train_data),label = as.matrix(as.factor(train_label_mean)))

In [24]:
xgb_mean_reg <- xgboost(data = dtrain_xg_mean_reg, nround=100)

[1]	train-rmse:1.482984 
[2]	train-rmse:1.222674 
[3]	train-rmse:1.021352 
[4]	train-rmse:0.871509 
[5]	train-rmse:0.776827 
[6]	train-rmse:0.683682 
[7]	train-rmse:0.634657 
[8]	train-rmse:0.551960 
[9]	train-rmse:0.507641 
[10]	train-rmse:0.451906 
[11]	train-rmse:0.424936 
[12]	train-rmse:0.384516 
[13]	train-rmse:0.357336 
[14]	train-rmse:0.344232 
[15]	train-rmse:0.335292 
[16]	train-rmse:0.317254 
[17]	train-rmse:0.306985 
[18]	train-rmse:0.291793 
[19]	train-rmse:0.261428 
[20]	train-rmse:0.248149 
[21]	train-rmse:0.236972 
[22]	train-rmse:0.215793 
[23]	train-rmse:0.200809 
[24]	train-rmse:0.190610 
[25]	train-rmse:0.176634 
[26]	train-rmse:0.172592 
[27]	train-rmse:0.167611 
[28]	train-rmse:0.161412 
[29]	train-rmse:0.148339 
[30]	train-rmse:0.142469 
[31]	train-rmse:0.138353 
[32]	train-rmse:0.134539 
[33]	train-rmse:0.129002 
[34]	train-rmse:0.121138 
[35]	train-rmse:0.113563 
[36]	train-rmse:0.109033 
[37]	train-rmse:0.100473 
[38]	train-rmse:0.094836 
[39]	train-rmse:0.089

In [25]:
xgb_mean_cl <- xgboost(data = dtrain_xg_mean_cl, nround=100, objective='multi:softmax',num_class=5)

[1]	train-mlogloss:1.330582 
[2]	train-mlogloss:1.109359 
[3]	train-mlogloss:0.954411 
[4]	train-mlogloss:0.844891 
[5]	train-mlogloss:0.741879 
[6]	train-mlogloss:0.664799 
[7]	train-mlogloss:0.593918 
[8]	train-mlogloss:0.527154 
[9]	train-mlogloss:0.481228 
[10]	train-mlogloss:0.424689 
[11]	train-mlogloss:0.397847 
[12]	train-mlogloss:0.358939 
[13]	train-mlogloss:0.331582 
[14]	train-mlogloss:0.310407 
[15]	train-mlogloss:0.288907 
[16]	train-mlogloss:0.271825 
[17]	train-mlogloss:0.251563 
[18]	train-mlogloss:0.230487 
[19]	train-mlogloss:0.218542 
[20]	train-mlogloss:0.203129 
[21]	train-mlogloss:0.190890 
[22]	train-mlogloss:0.178679 
[23]	train-mlogloss:0.168872 
[24]	train-mlogloss:0.158078 
[25]	train-mlogloss:0.147601 
[26]	train-mlogloss:0.137548 
[27]	train-mlogloss:0.129712 
[28]	train-mlogloss:0.122423 
[29]	train-mlogloss:0.115614 
[30]	train-mlogloss:0.109640 
[31]	train-mlogloss:0.103710 
[32]	train-mlogloss:0.098466 
[33]	train-mlogloss:0.094163 
[34]	train-mlogloss

In [26]:
lgb_mean_reg <- lgb.train(data = dtrain_lg_mean_reg,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.251791 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1515
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 18
[LightGBM] [Info] Start training from score 1.709898
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [27]:
params <- list(objective = "multiclass",
               num_class = 5, 
               metric = "multi_logloss")
lgb_mean_cl <- lgb.train(data = dtrain_lg_mean_cl,params=params,nrounds = 100)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.303892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1515
[LightGBM] [Info] Number of data points in the train set: 293, number of used features: 18
[LightGBM] [Info] Start training from score -1.225825
[LightGBM] [Info] Start training from score -1.873510
[LightGBM] [Info] Start training from score -1.505785
[LightGBM] [Info] Start training from score -1.553038
[LightGBM] [Info] Start training from score -2.124825
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [28]:
end_time = Sys.time()
time_matrix[3,]=end_time-start_time

## predict

In [37]:
start_time = Sys.time()

In [38]:
alldatalgb <- lgb.Dataset(data = as.matrix(data))
alldataxgb <- xgb.DMatrix(data = as.matrix(data))


In [39]:
xgbregmin=predict(xgb_min_reg,alldataxgb)
xgbclsmin=predict(xgb_min_cl,alldataxgb)
lgbregmin=predict(lgb_min_reg,as.matrix(data))
lgbclsmin=predict(lgb_min_cl,as.matrix(data))

In [40]:
xgbregmin

[1]  2.994804e+00  1.005534e+00  1.000981e+00  2.000845e+00  4.000088e+00
  [6]  4.000863e+00  2.317517e+00  2.999913e+00  9.994506e-01  5.049348e-05
 [11]  3.998717e+00  2.816837e+00  3.999374e+00  3.999627e+00  2.814681e+00
 [16]  8.375195e-03  3.996884e+00  3.000292e+00  3.788400e-03  2.998706e+00
 [21]  4.000552e+00  2.295758e+00  2.002082e+00  2.934469e+00  7.897200e-04
 [26]  9.980074e-01  4.301996e-01  2.350003e+00  3.828799e-04  1.108034e-03
 [31]  3.998085e+00  1.534405e+00 -8.303411e-04  2.929513e+00  5.315089e-04
 [36]  7.189436e-01  3.998626e+00  1.357631e-03 -3.279069e-01  8.757762e-04
 [41]  6.511146e-03  3.498876e+00  2.532552e+00  1.983810e+00  2.997890e+00
 [46]  3.148831e-03  2.655222e+00  1.638052e+00  3.999662e+00  1.000308e+00
 [51]  2.680547e-03  2.993995e+00  3.998460e+00  3.998013e+00  3.993052e+00
 [56]  2.999886e+00  3.999804e+00  8.973671e-04  2.000014e+00  4.256412e-03
 [61]  2.999075e+00 -2.178215e-04  1.846088e+00  3.998187e+00  1.998426e+00
 [66]  3.998446e+00  3.001913e+00  3.098105e-03 -5.873843e-05  3.999175e+00
 [71]  3.992883e-04  3.994661e+00  3.998813e+00  1.991780e+00  1.157595e-03
 [76]  2.998506e+00  2.164907e+00  3.169936e+00  8.564875e-01  9.981271e-01
 [81]  9.997218e-01  3.998813e+00  2.208746e+00  2.840349e+00  1.001127e+00
 [86]  2.626276e-03  2.383365e+00  2.001148e+00  2.003866e+00  2.274194e+00
 [91]  2.219136e+00  1.997514e+00  2.999105e+00  9.776643e-04  8.362690e-02
 [96]  1.546532e-03  2.000568e+00  1.999544e+00  2.998471e+00  2.731716e+00
[101]  3.999574e+00  2.323193e-03  1.707744e-03  2.995382e+00  2.000079e+00
[106]  3.999866e+00  1.912830e+00  1.131215e+00  2.275575e-03 -1.635948e-04
[111]  1.419692e+00  3.273100e+00  4.000435e+00  9.936173e-01  2.232299e+00
[116]  3.000835e+00  1.549438e+00  1.954172e+00  2.999038e+00  2.458397e+00
[121]  4.000297e+00  2.001001e+00  1.998590e+00  2.000610e+00  1.996778e+00
[126]  3.000153e+00  1.927149e+00  1.257490e+00  1.003036e+00  5.188987e-04
[131]  2.382117e-04  1.118771e+00  1.744261e+00  4.001048e+00  4.000867e+00
[136]  1.000993e+00 -1.197472e-03  2.002310e+00  3.598309e-03  9.971406e-01
[141]  9.993976e-01  2.003610e+00  3.996713e+00  2.066913e+00  9.966075e-01
[146]  1.089069e+00  2.047666e+00  9.992146e-01  4.542276e-03  6.538178e-04
[151] -4.526835e-04  2.290581e-01  4.000028e+00  3.001773e+00  3.995789e+00
[156]  1.543802e+00  1.186887e+00  1.048515e+00  1.372298e-03  2.001468e+00
[161]  1.795542e+00  3.999799e+00  3.997829e+00  3.993899e+00  4.005799e+00
[166]  3.997513e+00  4.001046e+00  2.046117e+00  2.473114e+00  1.339677e+00
[171]  2.999130e+00  2.999459e+00  2.000496e+00  2.819455e+00  2.359156e+00
[176]  1.000566e+00  2.710637e+00  8.014415e-05  2.000199e+00  3.000454e+00
[181]  2.000905e+00  1.470523e+00  1.514884e+00  3.000654e+00  4.000491e+00
[186] -5.352168e-04  1.598875e-03  3.999850e+00  4.647977e-04  1.802761e-01
[191]  2.695651e+00  3.996983e+00  2.005384e+00  4.825256e-01  1.999774e+00
[196]  3.998730e+00  1.999008e+00  2.000930e+00  2.999590e+00  1.471359e+00
[201]  2.999161e+00  2.170520e+00  7.173126e-01  1.806414e+00  9.989728e-01
[206]  2.168531e+00  2.477457e+00  2.004775e+00  6.793657e-04  9.997801e-01
[211]  1.002183e+00  2.428567e+00  3.000623e+00  1.999802e+00  4.000498e+00
[216]  3.993619e+00  2.884411e+00  1.353548e+00  3.000603e+00  5.275232e-04
[221]  5.659108e-04  3.998930e+00  3.998151e+00  3.297446e+00  2.000781e+00
[226]  3.995491e+00  1.000010e+00  3.999492e+00  9.994439e-01  2.000696e+00
[231]  1.002857e+00  9.995452e-01  1.335723e+00  3.000800e+00  7.219415e-03
[236]  2.998782e+00  2.306682e-03  1.545465e+00  2.824197e+00  1.929567e+00
[241]  1.722776e-04  2.241038e+00  3.999308e+00  1.853580e+00  1.999063e+00
[246]  2.693050e+00  1.999113e-03  3.001534e+00  3.996757e+00  1.003597e+00
[251] -7.980353e-05  1.202959e+00  1.999364e+00  3.998862e+00  2.192473e+00
[256]  1.239841e+00  3.997765e+00  4.000062e+00  3.000779e+00  2.241951e+00
[261]  3.999399e+00  1.537680e+00 -1.839225e-05  1

In [41]:
xgbclsmin

[1] 3 1 1 2 4 4 4 3 1 0 4 3 4 4 3 0 4 3 0 3 4 0 2 2 0 1 0 4 0 0 4 4 0 4 0 1 4
 [38] 0 0 0 0 4 3 0 3 0 3 4 4 1 0 3 4 4 4 3 4 0 2 0 3 0 2 4 2 4 3 0 0 4 0 4 4 0
 [75] 0 3 0 4 3 1 1 4 4 4 1 0 4 2 2 0 3 2 3 0 0 0 2 2 3 3 4 0 0 3 2 4 0 3 0 0 2
[112] 4 4 1 3 3 2 2 3 3 4 2 2 2 2 3 2 1 1 0 0 1 1 4 4 1 0 2 0 1 1 2 4 2 1 1 1 1
[149] 0 0 0 1 4 3 4 1 3 4 0 2 2 4 4 4 4 4 4 4 4 1 3 3 2 2 4 1 3 0 2 3 2 2 4 3 4
[186] 0 0 4 0 0 4 4 2 2 2 4 2 2 3 2 3 2 2 2 1 2 4 2 0 1 1 2 3 2 4 4 4 4 3 0 0 4
[223] 4 2 2 4 1 4 1 2 1 1 4 3 0 3 0 0 2 2 0 3 4 2 2 3 0 3 4 1 0 2 2 4 2 2 4 4 3
[260] 4 4 3 0 2 0 0 4 2 0 2 4 4 2 2 0 3 3 1 0 3 2 4 3 4 2 4 4 2 3 3 0 4 4 2 0 0
[297] 0 4 3 4 2 3 0 0 1 2 1 4 2 2 2 2 0 0 3 0 2 2 3 3 4 2 2 0 3 2 2 4 1 1 3 1 3
[334] 3 4 0 4 0 0 4 4 0 4 0 0 0 4 0 3 4 3 0 3 4 3 4 0 3 1 0 4 0 3 3 4 4 3 1 2 4
[371] 0 2 0 4 2 2 4 1 0 0 0 2 1 1 0 3 0 0 1 0 2 2 3 4 2 3 0 1 4 4 1 1 2 3 0 4 1
[408] 0 0 1 1 1 4 4

In [42]:
lgbregmin

[1]  1.620273497  1.248783769  1.326972202  2.482048823  3.611135882
  [6]  3.732004011  2.910237273  3.032919730  1.265540579  0.600268725
 [11]  3.742657052  3.160072667  3.387395583  3.058167057  2.934049299
 [16]  0.730618742  3.432098038  2.904357207  0.384433649  3.064092012
 [21]  3.893099255  0.918458533  1.853638916  3.025389050  0.438560073
 [26]  0.845805034  0.790488994  3.026545002  0.437777573  0.570906995
 [31]  2.971774275  2.775041334  0.306014831  1.296480843  0.869951797
 [36]  1.306103820  2.563918247  0.426514928  1.249250975  0.671647965
 [41]  0.898716367  3.478630207  2.784419090  1.746765907  2.317788996
 [46]  0.288045708  2.332486943  2.685808735  3.728829559  0.964415982
 [51]  0.827102665  2.624155172  3.607210259  3.513737522  2.593288746
 [56]  2.428139952  3.773250444  0.637434808  1.996096646  0.329147157
 [61]  2.527362425 -0.099827208  1.628743920  3.822812436  1.788515409
 [66]  3.344992055  3.034364712  0.332330851  0.685945441  3.814781013
 [71]  0.512892200  3.209635683  3.464429006  1.454894686  0.425067874
 [76]  2.650543695  3.024127550  3.132627813  2.721996969  0.790134606
 [81]  1.191441050  3.794117681  1.806078458  2.138730154  1.322796491
 [86]  1.002150497  2.684026568  2.553028803  2.282200412  2.420218229
 [91]  1.985331831  2.378126892  2.250704012  1.068790093  1.149498521
 [96]  0.617223957  2.408792802  2.148384227  3.195171877  2.228482022
[101]  3.340663015  0.456214168  0.188835637  2.069593690  1.899670812
[106]  3.452025085  1.725807805  2.052530365  0.595821999  0.932270286
[111]  1.905129634  3.890290588  3.171707803  0.637061491  2.046477741
[116]  2.657898285  1.381522491  2.026702711  2.562885107  1.471286498
[121]  3.735810172  2.362337536  1.575060623  1.724552765  1.914695104
[126]  2.981780497  3.107263838  1.939155189  0.866749212  0.668432539
[131]  0.482557466  0.361201324  1.733289832  3.709373539  3.374316744
[136]  1.506135140  0.528155626  2.255161796  0.655525234  1.022864582
[141]  0.965515390  2.014428790  3.872853914  2.531407007  0.825283156
[146]  2.105135091  1.709763449  1.276003538  0.892290782  0.595871509
[151]  0.332328532  0.990834208  3.992636430  3.397178319  3.109706990
[156]  0.903075777  2.785804251  2.516553371  0.503273249  2.483766987
[161]  3.095242189  3.090915889  3.482406482  3.538011613  4.140930418
[166]  3.666758137  3.836334615  2.266491344  3.194402486  2.562056008
[171]  2.844895414  2.523005131  2.256434365  2.975675024  3.503249002
[176]  1.193607034  2.855643402  0.054509307  2.272529595  2.658274468
[181]  2.210208432  2.924804081  3.399750736  3.096710406  3.567192986
[186]  0.227308245  0.687042517  3.936602601  0.107157340  0.622919994
[191]  3.778887833  3.719654003  2.514296716  2.243305256  1.990413998
[196]  3.399089376  1.551495076  2.649374728  2.897869304  1.800747266
[201]  2.356573877  3.103540355  1.225193076  2.360227628  1.395765650
[206]  2.225942839  3.652460999  2.088330121  0.340282861  1.172073596
[211]  1.271566402  3.572833547  2.890602005  1.553270040  3.544273142
[216]  3.208593217  4.120186256  2.772761003  2.803471706  0.480495287
[221]  0.206806244  3.873580920  3.443936299  3.927412904  2.447076857
[226]  3.713035326  1.363466252  3.560976251  0.440970859  2.284129200
[231]  1.619288828  1.112482306  2.604978058  2.715057022  1.150672431
[236]  3.323355955  0.577017810  1.617817750  2.898279806  3.132220358
[241]  0.703043349  3.348195648  3.493660836  1.709244938  1.193204855
[246]  2.669814413  1.467816466  3.089983409  3.618343422  1.431537088
[251]  0.544980053  2.674914175  1.244868339  3.476237941  3.357232219
[256]  2.454378274  3.951372020  3.632945724  2.678516920  3.493721736
[261]  3.799009151  2.580841616  0.261106531  2.822528541  2.008031851
[266]  2.090408473  3.591461301  2.041164265  1.799441176  2.964391384
[271]  4.213727483  3.242638288  2.529047699  2.091131582  0.439006366
[276]  1.978911543  2.992017842  0.779920856  0.277348473  2.971062819
[281]  1.921513948  3.7215

In [43]:
lgbclsmin

0.051712290,0.0018425021,0.0002452018,0.9447865587,0.001413448
0.020242740,0.9728644540,0.0003668366,0.0041881117,0.002337858
0.011899312,0.9581767278,0.0002177941,0.0011170301,0.028589136
0.001857566,0.0005944868,0.9411745833,0.0046213982,0.051751966
0.010487815,0.0023811602,0.0140683932,0.0027699335,0.970292699
0.014846089,0.0009306686,0.0037044533,0.0021489290,0.978369860
0.035444925,0.0039035692,0.0129136532,0.0061636585,0.941574194
0.012260263,0.0005768918,0.0019836976,0.9589694705,0.026209677
0.012808098,0.9724056863,0.0023063920,0.0039242900,0.008555534
0.962603073,0.0011445048,0.0079567163,0.0180805084,0.010215198
0.005212174,0.0050865226,0.0016955805,0.0173120976,0.970693625


In [44]:
datalength=dim(data)[1]
lgbclsm=matrix(lgbclsmin,m,datalength)
lgbclsminr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsminr[i,]=which.max(lgbclsm[,i])
}
lgbclsminr=lgbclsminr-1

In [45]:
lgbclsminr

0
4
2
0
4
3
2
2
0
0
0


In [46]:
xgbregmean=predict(xgb_mean_reg,alldataxgb)
xgbclsmean=predict(xgb_mean_cl,alldataxgb)
lgbregmean=predict(lgb_mean_reg,as.matrix(data))
lgbclsmean=predict(lgb_mean_cl,as.matrix(data))

In [47]:
lgbclsm=matrix(lgbclsmean,m,datalength)
lgbclsmeanr=matrix(0,datalength,1)
for(i in 1:datalength)
    {
    lgbclsmeanr[i,]=which.max(lgbclsm[,i])
}
lgbclsmeanr=lgbclsmeanr-1

In [48]:
preallmin=cbind(xgbclsmin,xgbregmin)
preallmin=cbind(preallmin,lgbclsminr)
preallmin=cbind(preallmin,lgbregmin)

In [49]:
colnames(preallmin)=c('xgbclsmin','xgbregmin','lgbclsmin','lgbregmin')
head(preallmin)

xgbclsmin,xgbregmin,lgbclsmin,lgbregmin
3,2.994804,0,1.620273
1,1.005534,4,1.248784
1,1.000981,2,1.326972
2,2.000845,0,2.482049
4,4.000088,4,3.611136
4,4.000863,3,3.732004


In [50]:
preallmean=cbind(xgbclsmean,xgbregmean)
preallmean=cbind(preallmean,lgbclsmeanr)
preallmean=cbind(preallmean,lgbregmean)

In [51]:
head(preallmean)

xgbclsmean,xgbregmean,,lgbregmean
2,1.995903015,2,1.1808193
1,0.996144235,4,0.6752915
0,0.002976565,4,0.3815441
0,0.004042335,3,0.6417456
2,1.997150064,4,1.9154230
3,2.999905348,0,2.4385762


In [52]:
colnames(preallmean)=c('xgbclsmean','xgbregmean','lgbclsmean','lgbregmean')
head(preallmean)

xgbclsmean,xgbregmean,lgbclsmean,lgbregmean
2,1.995903015,2,1.1808193
1,0.996144235,4,0.6752915
0,0.002976565,4,0.3815441
0,0.004042335,3,0.6417456
2,1.997150064,4,1.9154230
3,2.999905348,0,2.4385762


In [53]:
end_time = Sys.time()
time_matrix[4,]=end_time-start_time

In [54]:
result_list=list(preallmin,preallmean,time_matrix)

In [55]:
save(result_list, file = "Hourly_ets_opt_pre_result.RData")

In [56]:
time_matrix

user_time,system_time,elapsed_time
0.7882986,0.7882986,0.7882986
5.3128311,5.3128311,5.3128311
5.5616265,5.5616265,5.5616265
0.7945738,0.7945738,0.7945738


In [57]:
# 计算特征重要性
importance_matrix <- xgb.importance(model = xgb_min_cl)

# 打印特征重要性
print(importance_matrix)

              Feature        Gain       Cover   Frequency
               <char>       <num>       <num>       <num>
 1:         curvature 0.082801446 0.078685382 0.103822558
 2:         linearity 0.077036441 0.073251040 0.057574328
 3:           e_acf10 0.076377251 0.091135374 0.069844266
 4:           x_acf10 0.072072660 0.066413260 0.065361019
 5:       diff2_acf10 0.065253371 0.080064780 0.086833412
 6:         user_time 0.062778616 0.068675369 0.083294007
 7:        diff1_acf1 0.056314485 0.068335474 0.051439358
 8:            x_acf1 0.055381800 0.046393753 0.042000944
 9:           entropy 0.054967764 0.051630410 0.060877773
10:             spike 0.054793465 0.034613008 0.042236904
11:            e_acf1 0.053602627 0.049339280 0.050259556
12:        diff2_acf1 0.049515726 0.053304342 0.055214724
13:         seas_acf1 0.048916744 0.050372875 0.046720151
14: seasonal_strength 0.046802750 0.050169866 0.039641340
15:             trend 0.044350966 0.031551801 0.043416706
16:       diff

In [29]:
importance_matrix1 <- xgb.importance(model = xgb_mean_cl)
importance_matrix2 <- xgb.importance(model = xgb_mean_reg)
importance_matrix3 <- lgb.importance(lgb_mean_cl)
importance_matrix4 <- lgb.importance(lgb_mean_reg)

importance_matrix1=importance_matrix1[order(importance_matrix1$Feature),]
importance_matrix2=importance_matrix2[order(importance_matrix2$Feature),]
importance_matrix3=importance_matrix3[order(importance_matrix3$Feature),]
importance_matrix4=importance_matrix4[order(importance_matrix4$Feature),]

importance_matrix_mean=(importance_matrix1[,2:4]+importance_matrix2[,2:4]+importance_matrix3[,2:4]+importance_matrix4[,2:4])/4

importance=cbind(importance_matrix1[,1],importance_matrix_mean)

write.csv(importance,'h_ets_imp_mean.csv')